In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from spectral_cube import SpectralCube
from scipy.optimize import curve_fit

# === Step 1: Load ALMA Spectral Data (FITS File) ===
fits_file = "alma_spectrum.fits"  # Replace with actual ALMA FITS file
cube = SpectralCube.read(fits_file)
spectrum = cube.mean(axis=(1,2))  # Average over spatial region
frequencies = cube.spectral_axis.to('GHz').value  # Convert to GHz
intensities = spectrum.value  # Flux density in Jy/beam

# === Step 2: Define Gaussian Fitting Function ===
def gaussian(f, a, f0, sigma):
    return a * np.exp(- (f - f0)**2 / (2 * sigma**2))

# === Step 3: Identify Spectral Lines (Manually Input Known Transitions) ===
methanol_transitions = {
    "Line 1": {"freq": 96.741375, "E_u": 6.87, "g_u": 2, "A_ul": 3.69e-6},
    "Line 2": {"freq": 157.27092, "E_u": 15.2, "g_u": 4, "A_ul": 1.23e-5},
    # Add more transitions if needed
}

# === Step 4: Fit Spectral Lines ===
integrated_intensities = []
frequencies_fitted = []

for line, params in methanol_transitions.items():
    f0 = params["freq"]
    idx = np.argmin(np.abs(frequencies - f0))  # Find closest observed frequency
    f_range = frequencies[idx-10:idx+10]
    i_range = intensities[idx-10:idx+10]

    popt, _ = curve_fit(gaussian, f_range, i_range, p0=[max(i_range), f0, 0.001])
    a, f_fit, sigma = popt
    integrated_intensity = a * sigma * np.sqrt(2 * np.pi)  # Gaussian integral

    integrated_intensities.append(integrated_intensity)
    frequencies_fitted.append(f_fit)
    methanol_transitions[line]["N_u"] = integrated_intensity  # Store N_u

# === Step 5: Compute Column Densities (N_u) ===
for line, params in methanol_transitions.items():
    nu = params["freq"] * 1e9  # Convert GHz to Hz
    A_ul = params["A_ul"]
    g_u = params["g_u"]
    
    N_u = (8 * np.pi * 1.38e-23 * nu**2) / (6.63e-34 * (3e8)**3 * A_ul) * params["N_u"]
    methanol_transitions[line]["N_u"] = N_u

# === Step 6: Plot Population Diagram ===
E_u_values = np.array([params["E_u"] for params in methanol_transitions.values()])
N_u_values = np.array([params["N_u"] for params in methanol_transitions.values()])
g_u_values = np.array([params["g_u"] for params in methanol_transitions.values()])

ln_Nu_over_gu = np.log(N_u_values / g_u_values)

plt.figure(figsize=(8, 6))
plt.scatter(E_u_values, ln_Nu_over_gu, color='blue', label="Methanol Transitions")
plt.xlabel(r"$E_u$ (K)")
plt.ylabel(r"$\ln(N_u / g_u)$")
plt.title("Methanol Population Diagram (ALMA)")
plt.grid(True)
plt.legend()
plt.show()
